In [1]:
import svgBlockLib

from jupyter_cadquery import (
    PartGroup, Part, Edges, Faces, Vertices, show,
    close_viewer, close_viewers, get_viewer, open_viewer, set_defaults, get_defaults
)

from jupyter_cadquery.replay import replay, enable_replay, disable_replay, reset_replay

set_defaults(axes=True, timeit=False, show_parent=False)

enable_replay(False, False)
show_object = replay

cv = open_viewer("3dViewer", anchor="right", height=1024)

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


In [2]:
blk = svgBlockLib.blkLibrary()

In [3]:
#Import from file. Could be web or others in the future
#fileName = 'noun-gutenberg-press-86670-cleaned.svg'
#fileName = 'noun-houseplants-886781-edited.svg'
fileName = 'noun-ornaments-714185-cleaned.svg'
#fileName = 'maliaBirthday/noun-queen-4902988.svg'
#fileName = 'maliaBirthday/noun-corner-ornamental-6231875.svg'
#fileName = 'noun-barn-6921172.svg'

blk.readSVGFromFile(fileName)

#Parse using svgpathtools and 'return' paths and attribs.
#Could probably have some checks to make sure paths exist and are closed?
blk.parseSVG()

#See how many paths were parsed
print('Paths Found: %s' % format(blk.pathCount))

#Get Estimated width and height
blk.estimateSVGSize()
print('Estimated Width: {:.2f} - Estimated Height: {:.2f}'.format(blk.estimatedWidth, blk.estimatedHeight))

#Skip paths. Helpful for testing or omitting paths. 
#Needs to be tested
#blk.skipPathNumber.append([100,200])

#Get largest w/h value and set neckHeight before extruding
# Also sets the scale for use later on
if max(blk.estimatedHeight, blk.estimatedWidth) < 100:
    print('A')
    blk.neckHeight = 2
    blk.setScale(1)
    blk.feetCutOutPercentage = 0.08
    blk.filletAmount = 0.02
elif max(blk.estimatedHeight, blk.estimatedWidth) < 1000:
    print('B')
    blk.neckHeight = 20
    blk.setScale(10)
    blk.feetCutOutPercentage = 0.10
    blk.filletAmount = 0.2
elif max(blk.estimatedHeight, blk.estimatedWidth) < 10000:
    print('C')
    blk.neckHeight = 200
    blk.setScale(100)
    blk.feetCutOutPercentage = 0.12
    blk.filletAmount = 2
else:
    print('D')
    blk.neckHeight = 2000
    blk.setScale(1000)
    blk.feetCutOutPercentage = 0.14
    blk.filletAmount = 20

#Before translating from 2D to 3D, the neck height must be set
# Need to rethink as the scale isn't set yet. Circular dependency!
#blk.neckHeight = 2

#Do the actual translation from 2D to 3D
blk.translate2Dto3D()

#show_object(blk.base)

Paths Found: 1
826.9414306840708 353.5336921039335
Estimated Width: 826.94 - Estimated Height: 353.53
B


In [4]:
#Add some buffer space around the 2D SVG. Must be >1.0. 
#Must be called before building body
blk.xLenAdj = 1.02
blk.yLenAdj = 1.02

#Draw a box around the 3D SVG to get actual size.
blk.buildBoundingBox()

#With the box built, we can get our height and width
print('Block Width: ', blk.width/blk.scaleBy/25.4, "in")
print('Block Height', blk.height/blk.scaleBy/25.4, "in")

print(blk.width)
print(blk.height)

#Fillet outer edges. This value is usually close to the scale set below
# It is updated when setScale is called by multiplying filletAmount by
# the scale set
#blk.filletAmount = 0.01

#Parsing SVG will provide the Height/Width if listed.
#If the SVG is too large, the 3D block can be scaled. Ideally by factor of 10.
# Must call doMath to generate values
#blk.setScale(10)

#With the scale set, start calculating heights and widths
blk.doMath()

#Extrude basic block
blk.buildBody()

Block Width:  3.3207884232352463 in
Block Height 1.4197022289370553 in
843.4802595017526
360.604366150012


In [5]:
'''
This section pertains to the 3D portion of the process

Need to check if hollowBody function needs to create a second 
BoundingBox or if previous box can be used. This could allow for
x/yWallThickness to be defined earlier in the process.

feetCutOut is harder since it uses the X Face to calculate the
different values.
'''

#Adjust the x/y of the hollow box to be cut out. This is a percentage
# of the overal x/y of the block body. Must be < 1. Default: 0.8
# The smaller the number, the larger the hollow area
blk.xhollowPercentage = 0.8
blk.yhollowPercentage = 0.8

#
blk.hollowBody()
blk.createAndCutPyramid()

#Height of the feet cutout can be adjusted. This is in percentage
# of the total block height. This is usually a small number <0.2
#blk.feetCutOutPercentage = 0.10
blk.cutFeet()

#Smooth out the block edges a bit
blk.smoothOuterEdges()

#Export to STL file. If scale is anything by 1, it will scale the STL.
# STL only needs to be scaled in the Z direction. By default, it'll
# scale by XYZ.
#blk.exportSTL('corner')

#https://stackoverflow.com/questions/65241421/scaling-stl-mesh-object-in-python

In [34]:
#Export to SVG and display SVG name
#0,0,1 - top
#0,0,-1 - bottom
#0.5, 0.5, 0.5 - iso
#left: 1,0,0 but rotate

print( blk.exportSVG(projectionDir=(0, 1, 1),
                     strokeColor=(0,0,0),
                     strokeWidth=1.6,
                     hiddenColor=(120,120,120),
                     #includeGrid=True,
                     #view_vector=(0.612375, 0.612375, -0.5) 
                    ))

(0, 1, 1)
{'width': 600, 'height': 400, 'marginLeft': 60, 'marginTop': 60, 'showAxes': False, 'projectionDir': (0, 1, 1), 'strokeWidth': 1.6, 'strokeColor': (0, 0, 0), 'hiddenColor': (120, 120, 120), 'showHidden': True}
svgs/59449272-379d-4f99-8329-0c4e4d5676b5.svg


In [7]:
show_object(blk.base)

Use the multi select box below to select one or more steps you want to examine
